In [1]:
import malaya
from glob import glob
from tqdm import tqdm
import json
import random

news = glob('news/*.json')
len(news)

974

In [2]:
with open('populate-news.json') as fopen:
    data = json.load(fopen)
    
data.keys()

dict_keys(['text', 'title'])

In [3]:
data['text'][0]

'Ikatan alumni bintara polisi angkatan ke-27 tahun 2005 gelombang II (ZLD) Polres Sekadau, melakukan aksi bakti sosial dengan membagikan paket sembako kepada warga kurang mampu dan panti asuhan. Sebanyak 25 paket sembako berisi beras, gula, tepung, minyak goreng dan mie instan diserahkan langsung secara door to door kepada warga yang sebelumnya telah didata, untuk menghindari kerumunan. Sasarannya adalah warga kurang mampu atau fakir miskin, lansia, warga terdampak pandemi covid-19 dan anak yatim piatu di panti asuhan Harapan Bunda dan Yayasan Filipi. Bripka Hisbullah Aji selaku koordinator aksi baksos mengatakan, ide pemberian bantuan sembako muncul dari rasa keprihatinan terhadap warga lantaran sulitnya ekonomi ditengah masa pandemi. Terlebih bagi warga yang belum menerima bantuan dari pemerintah. Anggota ZLD Polres Sekadau yang berjumlah 22 personel kemudian mengadakan iuran sukarela yang dikumpulkan sehingga aksi bakti sosial ini dapat terealisasi. Bahkan anggota ZLD yang sudah ber

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from malaya.text.vectorizer import SkipGramCountVectorizer

stopwords = malaya.text.function.get_stopwords()
bow = CountVectorizer(
    ngram_range = (1, 3),
    stop_words = stopwords,
    lowercase = False,
)

stopwords = malaya.text.function.get_stopwords()
skip_bow = SkipGramCountVectorizer(
    ngram_range = (1, 3),
    stop_words = stopwords,
    lowercase = False,
    skip = 2
)

In [5]:
rejected = ['saya', 'awak', 'kami']

def headline(string, length = 300):
    splitted = malaya.text.function.split_into_sentences(string)
    splitted = [s for s in splitted if all([r not in s for r in rejected])]
    selected, index, results = '', 0, []
    while len(selected) <= length and index < len(splitted):
        selected += splitted[index]
        results.append(splitted[index])
        index += 1
    return ' '.join(results)

In [6]:
headline(data['text'][0])

'Ikatan alumni bintara polisi angkatan ke-27 tahun 2005 gelombang II (ZLD) Polres Sekadau, melakukan aksi bakti sosial dengan membagikan paket sembako kepada warga kurang mampu dan panti asuhan. Sebanyak 25 paket sembako berisi beras, gula, tepung, minyak goreng dan mie instan diserahkan langsung secara door to door kepada warga yang sebelumnya telah didata, untuk menghindari kerumunan.'

In [7]:
from tqdm import tqdm

def loop(texts):
    texts = texts[0]
    results = []
    for i in tqdm(range(len(texts))):
        try:
            h = headline(texts[i])
            keywords = malaya.keyword_extraction.rake(texts[i], 
                                                      vectorizer = bow, 
                                                      top_k = random.randint(5, 10))
            keywords = [k[1] for k in keywords]
            keywords_rake = malaya.keyword_extraction.rake(texts[i],
                                                      top_k = random.randint(5, 10))
            keywords_rake = [k[1] for k in keywords_rake]
            results.append((keywords, keywords_rake, h))
        except:
            pass
        
    return results

In [8]:
import mp
results = mp.multiprocessing(data['text'], loop, cores = 15)

100%|██████████| 10448/10448 [08:35<00:00, 20.29it/s]


In [9]:
results[0]

(['ZLD Polres Sekadau',
  'Polres Sekadau aksi',
  'sembako warga panti',
  'Sekadau aksi bakti',
  'warga panti asuhan'],
 ['Ikatan alumni bintara polisi angkatan ke-27',
  'warga lantaran sulitnya ekonomi ditengah',
  'ide pemberian bantuan sembako muncul',
  '25 paket sembako berisi beras',
  'panti asuhan Harapan Bunda'],
 'Ikatan alumni bintara polisi angkatan ke-27 tahun 2005 gelombang II (ZLD) Polres Sekadau, melakukan aksi bakti sosial dengan membagikan paket sembako kepada warga kurang mampu dan panti asuhan. Sebanyak 25 paket sembako berisi beras, gula, tepung, minyak goreng dan mie instan diserahkan langsung secara door to door kepada warga yang sebelumnya telah didata, untuk menghindari kerumunan.')

In [10]:
with open('keywords-headline.json', 'w') as fopen:
    json.dump(results, fopen)